<a href="https://colab.research.google.com/github/soheilpaper/-tft-2.4-ili9341-STM32/blob/master/Watermark%20remover/split_then_refine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Colab contains the Official demo of our AAAI21 paper:

[**Split then Refine: Stacked Attention-guided ResUNets for Blind Single Image Visible Watermark Removal**](https://arxiv.org/abs/2012.07007)

Xiaodong Cun, Chi-Man Pun*

University of Macau

In [1]:
# download the necessary componments
! rm -rf *
! git clone https://github.com/vinthony/deep-blind-watermark-removal.git # get code from github
! gdown https://drive.google.com/uc?id=1KpSJ6385CHN6WlAINqB3CYrJdleQTJBc # get pretrained model
! gdown https://drive.google.com/uc?id=18HaWfYYZCD34VttSjd2at8b9BKdhgVgU && unzip -q val.zip # get validation dataset (2.31G) of 27kpng
! gdown https://drive.google.com/uc?id=1it5oQDRqRzBVieX6jKNmOxj1992f63yM && unzip -q natural.zip # get natural images (0.4G) of 27kpng

# rename natural images
from os import listdir
from os.path import isfile, join
import shutil
filenames = [ shutil.copy(join('./natural', f), join('./natural', f).split('-')[0]+'.jpg') for f in listdir('./natural') if isfile(join('./natural', f)) ]

Cloning into 'deep-blind-watermark-removal'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 92 (delta 13), reused 6 (delta 6), pack-reused 67 (from 1)
Receiving objects: 100% (92/92), 53.16 KiB | 756.00 KiB/s, done.
Resolving deltas: 100% (26/26), done.
Downloading...
From (original): https://drive.google.com/uc?id=1KpSJ6385CHN6WlAINqB3CYrJdleQTJBc
From (redirected): https://drive.google.com/uc?id=1KpSJ6385CHN6WlAINqB3CYrJdleQTJBc&confirm=t&uuid=d05d5b16-1103-4419-bb98-2ec0f74940ee
To: /content/27kpng_model_best.pth.tar
100% 131M/131M [00:03<00:00, 36.2MB/s]
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?i

In [3]:
import os, sys, torch,random
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm

sys.path.append('deep-blind-watermark-removal')
sys.path.insert(0,'deep-blind-watermark-removal')

from scripts.utils.imutils import im_to_numpy
import scripts.models as models
import scripts.datasets as datasets
%matplotlib inline
from PIL import Image, ImageChops

def get_jet():
    colormap_int = np.zeros((256, 3), np.uint8)

    for i in range(0, 256, 1):
        colormap_int[i, 0] = np.int_(np.round(cm.jet(i)[0] * 255.0))
        colormap_int[i, 1] = np.int_(np.round(cm.jet(i)[1] * 255.0))
        colormap_int[i, 2] = np.int_(np.round(cm.jet(i)[2] * 255.0))

    return colormap_int

def clamp(num, min_value, max_value):
    return max(min(num, max_value), min_value)

def gray2color(gray_array, color_map):

    rows, cols = gray_array.shape
    color_array = np.zeros((rows, cols, 3), np.uint8)

    for i in range(0, rows):
        for j in range(0, cols):
#             log(256,2) = 8 , log(1,2) = 0 * 8
            color_array[i, j] = color_map[clamp(int(abs(gray_array[i, j])*10),0,255)]

    return color_array

class objectview(object):
    def __init__(self, *args, **kwargs):
        d = dict(*args, **kwargs)
        self.__dict__ = d

jet_map = get_jet()

resume_path = '27kpng_model_best.pth.tar' # path of pretrained model
samples = [320,1364,1868] #random.sample(range(4000), 1) # show random sample

data_config  = objectview({'input_size':256,
                            'limited_dataset':0,
                            'normalized_input':False,
                            'data_augumentation':False,
                            'base_dir':'/content/deep-blind-watermark-removal',
                            'data':'_images'})

val_loader = torch.utils.data.DataLoader(datasets.COCO('val',config=data_config,sample=samples))

print('input          | target              | coarser            | final')
print('----------------------------------------------------------------------------')
print('predicted mask | predicted watermark | coarser difference | final difference')

with torch.no_grad():

      model = models.__dict__['vvv4n']().cuda()
      model.load_state_dict(torch.load(resume_path)['state_dict'])
      model.eval()

      for i, batches in enumerate(val_loader):

          plt.figure(figsize=(48,12))

          im,mask,target = batches['image'].cuda(),batches['mask'].cuda(),batches['target'].cuda()

          imoutput,immask,imwatermark = model(im)

          imcoarser,imrefine,imwatermark = imoutput[1]*immask + im*(1-immask),imoutput[0]*immask + im*(1-immask),imwatermark*immask

          ims1 = im_to_numpy(torch.clamp(torch.cat([im,target,imcoarser,imrefine],dim=3)[0]*255,min=0.0,max=255.0)).astype(np.uint8)

          imcoarser, imrefine, target  = im_to_numpy((imcoarser[0]*255)).astype(np.uint8), im_to_numpy((imrefine[0]*255)).astype(np.uint8), im_to_numpy((target[0]*255)).astype(np.uint8)
          immask, imwatermark = im_to_numpy((immask.repeat(1,3,1,1)[0]*255)).astype(np.uint8),im_to_numpy((imwatermark[0]*255)).astype(np.uint8)

          coarsenp = gray2color(np.array(ImageChops.difference(Image.fromarray(imcoarser),Image.fromarray(target)).convert('L')),jet_map)
          finenp = gray2color(np.array(ImageChops.difference(Image.fromarray(imrefine),Image.fromarray(target)).convert('L')),jet_map)

          imfinal = np.concatenate([ims1,np.concatenate([immask,imwatermark,coarsenp,finenp],axis=1)],axis=0)

          plt.imshow(imfinal,vmin=0.0,vmax=255.0)

FileNotFoundError: [Errno 2] No such file or directory: '/content/deep-blind-watermark-removal/val_images/image'

In [5]:
! gdown https://drive.google.com/uc?id=1KpSJ6385CHN6WlAINqB3CYrJdleQTJBc # get pretrained model
! gdown https://drive.google.com/uc?id=18HaWfYYZCD34VttSjd2at8b9BKdhgVgU && unzip -q val.zip # get validation dataset (2.31G) of 27kpng
! gdown https://drive.google.com/uc?id=1it5oQDRqRzBVieX6jKNmOxj1992f63yM && unzip -q natural.zip # get natural images (0.4G) of 27kpng

Downloading...
From (original): https://drive.google.com/uc?id=1KpSJ6385CHN6WlAINqB3CYrJdleQTJBc
From (redirected): https://drive.google.com/uc?id=1KpSJ6385CHN6WlAINqB3CYrJdleQTJBc&confirm=t&uuid=2f4ccefd-4f2e-4758-a1c9-c92ba222b4c7
To: /content/27kpng_model_best.pth.tar
100% 131M/131M [00:01<00:00, 74.3MB/s]
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=18HaWfYYZCD34VttSjd2at8b9BKdhgVgU

but Gdown can't. Please check connections and permissions.
Downloading...
From (original): https://drive.google.com/uc?id=1it5oQDRqRzBVieX6jKNmOxj1992f63yM
From (redirected): https://drive.google.com/uc?id=1it5oQDRqRzBVieX6jKNmOxj1992f63yM&confirm=t&uuid=d5858d81-9794-4f3c-bd87-24e29378c347
To: /content/natural.zip
100

In [7]:
# watermark synthesis
import os
import random
import shutil
from PIL import Image
import numpy as np

def trans_paste(bg_img,fg_img,mask,box=(0,0)):
    fg_img_trans = Image.new("RGBA",bg_img.size)
    fg_img_trans.paste(fg_img,box,mask=mask)
    new_img = Image.alpha_composite(bg_img,fg_img_trans)
    return new_img,fg_img_trans

if os.path.isdir('dataset'):
    shutil.rmtree('dataset')

os.mkdir('dataset')
BASE_IMG_DIR = '/Users/oishii/Downloads/val2014/'
WATERMARK_DIR = 'logos' #1080
images = sorted([os.path.join(BASE_IMG_DIR,x) for x in os.listdir(BASE_IMG_DIR) if '.jpg' in x])
watermarks = sorted([os.path.join(WATERMARK_DIR,x).replace(' ','_') for x in os.listdir(WATERMARK_DIR) if '.png' in x])
# rename all the watermark from replace ' ' to '_'

random.shuffle(images)
random.shuffle(watermarks)

train_images = images[:int(len(images)*0.7)]
val_images = images[int(len(images)*0.7):int(len(images)*0.8)]
test_images = images[int(len(images)*0.8):]

train_wms = watermarks[:int(len(watermarks)*0.7)]
val_wms = watermarks[int(len(watermarks)*0.7):int(len(watermarks)*0.8)]
test_wms = watermarks[int(len(watermarks)*0.8):]

# save all the settings to file
names = ['train_images','val_images','test_images','train_wms','val_wms','test_wms']
lists = [train_images,val_images,test_images,train_wms,val_wms,test_wms]
dataset = dict(zip(names, lists))

for name,content in dataset.items():
    with open('dataset/%s.txt'%name,'w') as f:
        f.write("\n".join(content))

print('SAVE ALL THE SETTING')

for name, images in dataset.items():
    if 'images' not in name:
        continue
    # for each setting, synthesis the watermark
    # for each image, add X(X=6) watermark in differnet position, alpha,
    # save the synthesized image, watermark mask, reshaped mask,
    save_path = 'dataset/%s/'%name
    os.makedirs('%s/image'%(save_path))
    os.makedirs('%s/mask'%(save_path))
    os.makedirs('%s/wm'%(save_path))

    for img in images:
        im = Image.open(img).convert('RGBA')
        imw,imh = im.size

        for wmg in random.choices(dataset[name.replace('images','wms')],k=6):
            wm = Image.open(wmg.replace('_',' ')).convert("RGBA") # RGBA
            # get the mask of wm
            # data agumentation of wm
            wm = wm.rotate(angle=random.randint(0,360),expand=True) # rotate

            # make sure the
            imrw = random.randrange(int(0.4*imw),int(0.8*imw))
            imrh = random.randrange(int(0.4*imh),int(0.8*imh))
            wmsize = imrh if imrw > imrh else imrw
            wm = wm.resize((wmsize,wmsize),Image.BILINEAR)
            w,h = wm.size # new size

            box_left = random.randint(0,imw-w)
            box_upper = random.randint(0,imh-h)
            wmm = wm.copy()
            wm.putalpha(random.randint(int(255*0.4),int(255*0.8))) # alpha

            ims,wmc = trans_paste(im,wm,wmm,(box_left,box_upper))

            wmnp = np.array(wmc) # h,w,3
            mask = np.sum(wmnp,axis=2)>0
            mm = Image.fromarray(np.uint8(mask*255),mode='L')

            identifier = os.path.basename(img).split('.')[0] +'-'+os.path.basename(wmg).split('.')[0] + '.png'
            # save
            wmc.save('%s/wm/%s'%(save_path,identifier))
            ims.save('%s/image/%s'%(save_path,identifier))
            mm.save('%s/mask/%s'%(save_path,identifier))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/oishii/Downloads/val2014/'